# Pore Scale Models

Pore scale models are one of the more important facets of OpenPNM, but they can be a bit confusing at first, since they work 'behind-the-scenes'.  
They offer 3 main advantages, in order: 

1. They allow automatic regeneration of all dependent properties when something 'upstream' is changed.
1. A large library of pre-written models is included and they can be mixed together and their parameters edited to get a desired overall result.
1. The pore-scale model machinery was designed to allow easy use of custom written code for cases where a prewritten model is not available. 

The best way to explain their importance is via illustration.  

Consider a fluid flow simulation, where the hydraulic conductance is defined as:

$$ g_H = \frac{\pi R^4}{8 \mu L} $$

The viscosity can be predicted using a correlation:

$$ \mu = ? $$

Now say you want to re-run the simulation at different temperature.  This would require recalculating $\mu$, followed by updating the hydraulic conductance.

Using pore-scale models in OpenPNM allows for simple and reliable updating of these properties, for instance within a for-loop where temperature is being varied.  

## Using Existing Pore-Scale Models

OpenPNM includes a library of pre-written models, so let's start by using an existing model:

In [1]:
import numpy as np
np.random.seed(0)
import openpnm as op
pn = op.network.Demo(shape=[5, 5, 1], spacing=1e-4)

Now we need to define the gas phase diffusivity.  We can fetch the ``fuller`` model from the ``models`` library to do this, and attach it to an empty phase object:

In [4]:
water = op.phase.Phase(network=pn)
f = op.models.phase.viscosity.water_correlation
water.add_model(propname='pore.viscosity', model=f)

We have used the ``water_correlation`` model, which is a polynomial fit specfically to the properties of water as a function temperature (and salinity if relevant).

Next we need to define the hydraulic conductance, which is also available in the model libary:

In [6]:
f = op.models.physics.hydraulic_conductance.generic_hydraulic
water.add_model(propname='throat.hydraulic_conductance', model=f)

Lastly we can run the StokesFlow algorithm to get the flow rate across the domain for a given set of pressure boundary conditions:

In [14]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=1)
sf.set_value_BC(pores=pn.pores('right'), values=0)
sf.run()
print(sf.rate(pores=pn.pores('left')))

[1.46458816e-13]                                                                                                       


## Changing dependent properties

Now consider that you want to find the diffusion rate at higher temperature.  This requires recalculating the diffusion coefficient on ``air``, then updating the diffusive conductivity on ``phys``, and finally re-running the simulation.  Using pore-scale models this can be done as follows:

In [11]:
print('Viscosity before changing temperaure:', water['pore.viscosity'][0])
water['pore.temperature'] = 353.0
water.regenerate_models()
print('Viscosity after:', water['pore.viscosity'][0])

Viscosity before changing temperaure: 0.000893190947459112
Viscosity after: 0.000893190947459112


We can see that the viscosity decreased with temperature as expected.  We can also propagate this change to the diffusive conductance:

And lastly we can recalculate the flow rate:

In [13]:
sf.run()
print(sf.rate(pores=pn.pores('left')))

[1.46458816e-13]                                                                                                       


## Updating parameter on an existing model

It's also easy to change parameters of a model since they are all stored on the object (``water`` in this case), meaning you don't have to reassign a new model get new parameters (although that would work).  The models and their parameters are stored under the ``models`` attribute of each object.  This is a dictionary with each model stored under the key match the ``propname`` to which is was assigned. 

In [5]:
print('Diffusivity before changing parameter:', air['pore.diffusivity'][0])
air.models['pore.diffusivity']['vA'] = 15.9
air.regenerate_models()
print('Diffusivity after:', air['pore.diffusivity'][0])

Diffusivity before changing parameter: 2.067547836430591e-05
Diffusivity after: 2.0969679602405972e-05


## Replacing an existing model with another



Let's say for some reason that the Fuller model is not suitable.  It's easy to go 'shopping' in the models library to retrieve a new model and replace the existing one.  In the cell below we grab the Chapman-Enskog model and simply assign it to the same ``propname`` that the Fuller model was previously.

In [10]:
f = op.models.phase.viscosity.liquid_pure_ls
water.add_model(propname='pore.viscosity', model=f)
print('Diffusivity after:', water['pore.diffusivity'][0])

KeyError: 'molecular_weight'

Note that we don't need to explicitly call ``regenerate_models`` since this occurs automatically when a model is added.  We do however, have to regenerate ``phys`` object so it calculates the diffusive conductance with the new diffusivity:

In [7]:
phys.regenerate_models()
fd.reset()
fd.run()
print(fd.rate(pores=pn.pores('left')))

[1.03738023e-10]


## Creating Custom Models

Lastly, let's illustrate the ease with which a custom pore-scale model can be defined and used.  Let's create a very basic (and incorrect) model:

In [11]:
def new_diffusivity(target, A, B, 
                    temperature='pore.temperature', 
                    pressure='pore.pressure'):
    T = target[temperature]
    P = target[pressure]
    DAB = A*T**3/(P*B)
    return DAB

There are a few key points to note in the above code.  

1. Every model must accept a ``target`` argument since the ``regenerate_models`` mechanism assumes it is present.  The ``target`` is the object to which the model will be attached.  It allows for the looking up of necessary properties that should already be defined, like temperature and pressure.  Even if you don't use ``target`` within the function it is still required by the pore-scale model mechanism.  If it's presence annoys you, you can put a ``**kwargs`` at the end of the argument list to accept all arguments that you don't explicitly need.  
2. The input parameters should not be arrays (like an Np-long list of temperature values).  Instead you should pass the dictionary key of the values on the ``target``.  This allows the model to lookup the latest values for each property when ``regenerate_models`` is called.  This also enables openpnm to store the model parameters as short strings rather than large arrays.
3. The function should return either a scalar value or an array of Np or Nt length.  In the above case it returns a DAB value for each pore, depending on its local temperature and pressure in the pore.  However, if the ``temperature`` were set to ``'throat.temperature'`` and ``pressure`` to ``'throat.pressure'``, then the above function would return a DAB value for each throat and it could be used to calculate ``'throat.diffusivity'``.
4. This function can be placed at the top of the script in which it is used, or it can be placed in a separate file and imported into the script with ``from my_models import new_diffusivity``.

Let's add this model to our ``air`` phase and inspect the new values:

In [12]:
air.add_model(propname='pore.diffusivity',
              model=new_diffusivity,
              A=1e-6, B=21)
print(air['pore.diffusivity'])

[2.06722719e-05 2.06722719e-05 2.06722719e-05 2.06722719e-05
 2.06722719e-05 2.06722719e-05 2.06722719e-05 2.06722719e-05
 2.06722719e-05 2.06722719e-05 2.06722719e-05 2.06722719e-05
 2.06722719e-05 2.06722719e-05 2.06722719e-05 2.06722719e-05
 2.06722719e-05 2.06722719e-05 2.06722719e-05 2.06722719e-05
 2.06722719e-05 2.06722719e-05 2.06722719e-05 2.06722719e-05
 2.06722719e-05]
